### Soccermatics
#### Assignment 2
##### Instructions
* It is summer of 2018. The head scout of your club (the one which group you are in) is looking to sign a player to improve your defence. It doesn't need to be a defender. But it does need to be a player who will strength some aspect of your defensive play.
* She is interested in hearing about a variety of methods for scouting players, so she asks you to create a player radar. You should primarily use the Wyscout data, but can also use Transfermarkt to find the prices of players in summer 2018.
* You should do the following steps:
   1. Implement a number of defensive metrics based on statistical scouting (lesson 3) and valuing actions (lesson 4).
      * Four to six metrics is about right.
      * Create player radars for your own team's players in the position you aim to replace (one of goalkeeper, defender, midfielder, forward).
      * Write a simple non-technical text (at most one page) explaining your radars to the scout and what assumptions it makes.
      * Points will be awarded for the novelty of the metric, the clarity of the explanation and for the technical difficulty of the contribution. (12 points).
   2. Use one of the metrics on your radar to find a single player in another league (not the Premier league), who you would recommend signing.
      * Produce a top-10 list of players in this league for one of the metrics.
      * On this basis, choose a player you would recommend and create a player radar for him. (3 points)
   3. By midnight Friday 30th September send your report (steps 1 and 2) to the other members of your group.
      * On Monday the 3rd of October, you will each present a radar for your player chosen in stage 2 within your group.
      * Your presentation should be no more than 2 minutes.
      * After the presentations, you will have a group discussion comparing your choices.
      * You should write contrasting your own and the other reports and make a final recommendation on this basis (participation in meeting 2 points; final report 3 points)
   4. The final report (deadline Wednesday 5th October) should be 4 pages: roughly 2 pages for part 1, 1 page for part 2, and 1 page for part 3.
   
   
Submit a separate runnable piece of code (preferably in Python) which when put in the directory of the Wyscout data creates the radars. This code must be single file and only uses standard libraries (mplsoccer is fine) and should be runnable when put in the folder with Wyscout data.

I have chosen to scout for Liverpool FC, looking for a midfielder who can help improve defensive outcomes by evaluating performance across the following metrics:
(all posession adjusted per 90):
* Interceptions
* Defensive duel won via anticipation (1801 = successful, 601, 602 = anticipation) Anticipation (602) is used with Defensive Duel and Anticipated (601) is used with Offensive Duel. https://footballdata.wyscout.com/events-manual/
* Dribbled past


In [1]:
import pandas as pd
import numpy as np
import json

from tqdm import tqdm

from dataclasses import dataclass

import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf

import os
import pathlib
import warnings

from scipy import stats
from mplsoccer import PyPizza, FontManager

In [2]:
MINIMUM_90s = 5
TARGET_ROLE = 'Midfielder' # Goalkeeper, Defender, Midfielder, Forward

In [3]:
# incomplete but good enough?
# this data uses v2 API
# https://docs.google.com/document/d/11mNbM7kX2xKStQ1UCoBfUgYNxJ2b1Kgqrg_x8wC6JQo/edit
# https://apidocs.wyscout.com/
wyscout_tags = [
    (101, "goal"),
    (102, "own_goal"),
    (301, "assist"),
    (302, "key_pass"),
    (1901, "counter_attack"),
    (401, "left_foot"),
    (402, "right_foot"),
    (403, "head/body"),
    (1101, "direct"),
    (1102, "indirect"),
    (2001, "dangerous_ball_lost"),
    (2101, "blocked"),
    (801, "high"),
    (802, "low"),
    (1401, "interception"),
    (1501, "clearance"),
    (201, "opportunity"),
    (1301, "feint"),
    (1302, "missed_ball"),
    (501, "free_space_right"),
    (502, "free_space_left"),
    (503, "take_on_left"),
    (504, "take_on_right"),
    (1601, "sliding_tackle"),
    (601, "anticipated"),
    (602, "anticipation"),
    (1701, "red_card"),
    (1702, "yellow_card"),
    (1703, "second_yellow_card"),
    (1201, "position_goal_low_center"),
    (1202, "position_goal_low_right"),
    (1203, "position_goal_mid_center"),
    (1204, "position_goal_mid_left"),
    (1205, "position_goal_low_left"),
    (1206, "position_goal_mid_right"),
    (1207, "position_goal_high_center"),
    (1208, "position_goal_high_left"),
    (1209, "position_goal_high_right"),
    (1210, "position_out_low_right"),
    (1211, "position_out_mid_left"),
    (1212, "position_out_low_left"),
    (1213, "position_out_mid_right"),
    (1214, "position_out_high_center"),
    (1215, "position_out_high_left"),
    (1216, "position_out_high_right"),
    (1217, "position_post_low_right"),
    (1218, "position_post_mid_left"),
    (1219, "position_post_low_left"),
    (1220, "position_post_mid_right"),
    (1221, "position_post_high_center"),
    (1222, "position_post_high_left"),
    (1223, "position_post_high_right"),
    (901, "through"),
    (1001, "fairplay"),
    (701, "lost"),
    (702, "neutral"),
    (703, "won"),
    (1801, "accurate"),
    (1802, "not_accurate")
]

In [4]:
wyscout_tags_dict = dict(wyscout_tags)

In [5]:
def get_wyscout_data(file_name, sub_directory=None):

    if sub_directory:
        path = os.path.join(str(pathlib.Path().resolve().parents[0]), 'data', 'Wyscout', sub_directory, file_name)
    else:
        path = os.path.join(str(pathlib.Path().resolve().parents[0]), 'data', 'Wyscout', file_name)

    #open data
    with open(path) as f:
        data = json.load(f)
    df = pd.DataFrame(data)
    
    return df

In [6]:
df_competitions = get_wyscout_data('competitions.json')
df_competitions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    7 non-null      object
 1   wyId    7 non-null      int64 
 2   format  7 non-null      object
 3   area    7 non-null      object
 4   type    7 non-null      object
dtypes: int64(1), object(4)
memory usage: 408.0+ bytes


In [7]:
df_teams = get_wyscout_data('teams.json')
df_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   city          142 non-null    object
 1   name          142 non-null    object
 2   wyId          142 non-null    int64 
 3   officialName  142 non-null    object
 4   area          142 non-null    object
 5   type          142 non-null    object
dtypes: int64(1), object(5)
memory usage: 6.8+ KB


In [8]:
df_players = get_wyscout_data('players.json')

df_players['role_name'] = df_players['role'].apply(lambda x: x.get('name'))

df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3603 entries, 0 to 3602
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   passportArea           3603 non-null   object
 1   weight                 3603 non-null   int64 
 2   firstName              3603 non-null   object
 3   middleName             3603 non-null   object
 4   lastName               3603 non-null   object
 5   currentTeamId          3512 non-null   object
 6   birthDate              3603 non-null   object
 7   height                 3603 non-null   int64 
 8   role                   3603 non-null   object
 9   birthArea              3603 non-null   object
 10  wyId                   3603 non-null   int64 
 11  foot                   3603 non-null   object
 12  shortName              3603 non-null   object
 13  currentNationalTeamId  3603 non-null   object
 14  role_name              3603 non-null   object
dtypes: int64(3), object(1

In [9]:
league_countries = ['England', 'Spain', 'Italy', 'Germany', 'France'] # ignore internationals i.e. World Cup and EUROs

In [10]:
matches = {}

for country in league_countries:
    file_name = 'matches_' + country + '.json'
    matches[country] = get_wyscout_data(file_name, sub_directory='matches')

df_matches = pd.concat(matches).reset_index()
df_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   level_0        1826 non-null   object
 1   level_1        1826 non-null   int64 
 2   status         1826 non-null   object
 3   roundId        1826 non-null   int64 
 4   gameweek       1826 non-null   int64 
 5   teamsData      1826 non-null   object
 6   seasonId       1826 non-null   int64 
 7   dateutc        1826 non-null   object
 8   winner         1826 non-null   int64 
 9   venue          1826 non-null   object
 10  wyId           1826 non-null   int64 
 11  label          1826 non-null   object
 12  date           1826 non-null   object
 13  referees       1826 non-null   object
 14  duration       1826 non-null   object
 15  competitionId  1826 non-null   int64 
dtypes: int64(7), object(9)
memory usage: 228.4+ KB


In [11]:
events = {}

for country in tqdm(league_countries):
    file_name = 'events_' + country + '.json'
    events[country] = get_wyscout_data(file_name, sub_directory='events')

df_events = pd.concat(events).reset_index()
df_events.info()

100%|██████████| 5/5 [01:21<00:00, 16.39s/it]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3071395 entries, 0 to 3071394
Data columns (total 14 columns):
 #   Column        Dtype  
---  ------        -----  
 0   level_0       object 
 1   level_1       int64  
 2   eventId       int64  
 3   subEventName  object 
 4   tags          object 
 5   playerId      int64  
 6   positions     object 
 7   matchId       int64  
 8   eventName     object 
 9   teamId        int64  
 10  matchPeriod   object 
 11  eventSec      float64
 12  subEventId    object 
 13  id            int64  
dtypes: float64(1), int64(6), object(7)
memory usage: 328.1+ MB


In [12]:
minutes_per_game = {}

for country in league_countries:
    file_name = 'minutes_played_per_game_' + country + '.json'
    minutes_per_game[country] = get_wyscout_data(file_name, sub_directory='minutes_played')

df_minutes_per_game = pd.concat(minutes_per_game).reset_index()
df_minutes_per_game.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50587 entries, 0 to 50586
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   level_0         50587 non-null  object
 1   level_1         50587 non-null  object
 2   playerId        50587 non-null  int64 
 3   shortName       50587 non-null  object
 4   matchId         50587 non-null  int64 
 5   teamId          50587 non-null  int64 
 6   teamName        50587 non-null  object
 7   player_in_min   50587 non-null  int64 
 8   player_out_min  50587 non-null  int64 
 9   minutesPlayed   50587 non-null  int64 
 10  red_card        50587 non-null  int64 
dtypes: int64(7), object(4)
memory usage: 4.2+ MB


In [13]:
group_by = ['playerId', 'shortName'] # group_by = ['level_0', 'playerId', 'shortName']

df_minutes_season = df_minutes_per_game[group_by + ['minutesPlayed']].groupby(group_by).sum()
df_minutes_season

,,minutesPlayed
playerId,shortName,
33,J. Cillessen,93
36,T. Alderweireld,1238
38,D. Blind,382
45,G. van der Wiel,313
48,J. Vertonghen,3343
...,...,...
545811,Hugo Duro,24
551398,David Alba,63
564512,S. Sanogo,44


In [14]:
team_id_liverpool = df_teams[df_teams['name']=='Liverpool']['wyId'].to_numpy()[0]
team_id_liverpool

1612

In [15]:
df_liverpool_players = df_players[df_players['currentTeamId']==team_id_liverpool]
df_liverpool_players

,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId,role_name
667,"{'name': 'Serbia', 'id': '688', 'alpha3code': ...",82,Marko,,Gruji\u0107,1612,1996-04-13,191,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'Serbia', 'id': '688', 'alpha3code': ...",237268,right,M. Gruji\u0107,17322,Midfielder
686,"{'name': 'England', 'id': '0', 'alpha3code': '...",67,Jordan Brian,,Henderson,1612,1990-06-17,182,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'England', 'id': '0', 'alpha3code': '...",7964,right,J. Henderson,2413,Midfielder
736,"{'name': 'Belgium', 'id': '56', 'alpha3code': ...",87,Simon,,Mignolet,1612,1988-03-06,193,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Belgium', 'id': '56', 'alpha3code': ...",8514,right,S. Mignolet,5629,Goalkeeper
1191,"{'name': 'Brazil', 'id': '76', 'alpha3code': '...",76,Roberto Firmino,,Barbosa de Oliveira,1612,1991-10-02,181,"{'code2': 'FW', 'code3': 'FWD', 'name': 'Forwa...","{'name': 'Brazil', 'id': '76', 'alpha3code': '...",15808,right,Roberto Firmino,6380,Forward
1278,"{'name': 'Switzerland', 'id': '756', 'alpha3co...",72,Xherdan,,Shaqiri,1612,1991-10-10,169,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'Kosovo', 'id': '0', 'alpha3code': 'X...",49872,left,X. Shaqiri,6697,Midfielder
1319,"{'name': 'Germany', 'id': '276', 'alpha3code':...",91,Alisson Rams\u00e9s,,Becker,1612,1992-10-02,191,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Brazil', 'id': '76', 'alpha3code': '...",214220,left,Alisson,6380,Goalkeeper
1380,"{'name': 'England', 'id': '0', 'alpha3code': '...",69,Trent,,Alexander-Arnold,1612,1998-10-07,175,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'England', 'id': '0', 'alpha3code': '...",346101,right,T. Alexander-Arnold,2413,Defender
1719,"{'name': 'Egypt', 'id': '818', 'alpha3code': '...",72,Mohamed,,Salah Ghaly,1612,1992-06-15,175,"{'code2': 'FW', 'code3': 'FWD', 'name': 'Forwa...","{'name': 'Egypt', 'id': '818', 'alpha3code': '...",120353,left,Mohamed Salah,16129,Forward
1800,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",75,Sadio,,Man\u00e9,1612,1992-04-10,175,"{'code2': 'FW', 'code3': 'FWD', 'name': 'Forwa...","{'name': 'Senegal', 'id': '686', 'alpha3code':...",25747,right,S. Man\u00e9,19314,Forward
1894,"{'name': 'Croatia', 'id': '191', 'alpha3code':...",84,Dejan,,Lovren,1612,1989-07-05,188,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Croatia', 'id': '191', 'alpha3code':...",25393,right,D. Lovren,9598,Defender


In [16]:
df_events

,level_0,level_1,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,England,0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171
1,England,1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172
2,England,2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173
3,England,3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174
4,England,4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3071390,France,632802,10,Shot,"[{'id': 101}, {'id': 401}, {'id': 201}, {'id':...",405977,"[{'y': 65, 'x': 87}, {'y': 100, 'x': 100}]",2501065,Shot,19830,2H,2821.203869,100,253316980
3071391,France,632803,9,Save attempt,"[{'id': 101}, {'id': 1208}, {'id': 1802}]",25521,"[{'y': 0, 'x': 0}, {'y': 35, 'x': 13}]",2501065,Save attempt,3795,2H,2824.817116,91,253316978
3071392,France,632804,8,Simple pass,[{'id': 1801}],295761,"[{'y': 52, 'x': 50}, {'y': 47, 'x': 44}]",2501065,Pass,3795,2H,2888.383858,85,253316981
3071393,France,632805,8,Simple pass,[{'id': 1801}],249926,"[{'y': 47, 'x': 44}, {'y': 8, 'x': 41}]",2501065,Pass,3795,2H,2889.654402,85,253316982


In [17]:
group_by = ['eventId', 'subEventId', 'eventName', 'subEventName']

df_events_grouped = df_events[group_by + ['id']].groupby(group_by).count().sort_values('eventId', ascending=False)
df_events_grouped.columns = ['number_of_events']
df_events_grouped

number_of_events
eventId subEventId eventName               subEventName                             
10      100        Shot                    Shot                                40461
9       91         Save attempt            Save attempt                         6375
        90         Save attempt            Reflexes                            10192
8       83         Pass                    High pass                          123214
        80         Pass                    Cross                               58634
        81         Pass                    Hand pass                           13135
        82         Pass                    Head pass                           91194
        84         Pass                    Launch                              43303
        85         Pass                    Simple pass                       1207448
        86         Pass                    Smart pass                          28428
7       72         Others on the ball      Touch                              165296
        71         Others on the ball      Clearance                           53165
        70         Others on the ball      Acceleration                        24376
6                  Offside                                                      7821
5       51         Interruption            Whistle                               917
        50         Interruption            Ball out of the field              129179
4       40         Goalkeeper leaving line Goalkeeper leaving line              5779
3       36         Free Kick               Throw in                            80305
        34         Free Kick               Goal kick                           29725
        33         Free Kick               Free kick shot                       2038
        32         Free Kick               Free kick cross                      8134
        31         Free Kick               Free Kick                           43544
        30         Free Kick               Corner                              18181
        35         Free Kick               Penalty                               541
2       25         Foul                    Simulation                            108
        27         Foul                    Violent Foul                           74
        26         Foul                    Time lost foul                        205
        24         Foul                    Protest                               561
        23         Foul                    Out of game foul                      473
        22         Foul                    Late card foul                        300
        21         Foul                    Hand foul                            1941
        20         Foul                    Foul                                44293
1       12         Duel                    Ground defending duel              264552
        11         Duel                    Ground attacking duel              265456
        13         Duel                    Ground loose ball duel             143861
        10         Duel                    Air duel                           158186

In [18]:
def filter_by_event_or_sub_event(df=df_events, eventId=None, subEventId=None):

    df_filtered = df.copy(deep=True)
    
    if eventId:
        df_filtered = df_filtered[df_filtered['eventId']==eventId]

    if subEventId:
        df_filtered = df_filtered[df_filtered['subEventId']==subEventId]

    return df_filtered

In [19]:
def get_tags_for_event(eventId=None, subEventId=None):

    df_filtered = filter_by_event_or_sub_event(eventId=eventId, subEventId=subEventId)
        
    df_exploded = df_filtered.explode('tags')
    df_exploded['tag_id'] = df_exploded['tags'].apply(lambda x: x.get('id'))

    df_grouped = df_exploded[['tag_id', 'eventName', 'subEventName', 'eventId']].groupby(['eventName', 'subEventName', 'tag_id']).count()
    df_grouped = df_grouped.sort_values('eventId', ascending=False)
    df_grouped.columns = ['number_of_events']

    return df_grouped

In [20]:
get_tags_for_event(eventId=1, subEventId=12)

number_of_events
eventName subEventName          tag_id                  
Duel      Ground defending duel 1801              133399
                                1802              131153
                                701               130720
                                702                75284
                                703                58111
                                502                30149
                                501                29330
                                504                27143
                                503                22843
                                601                16358
                                1601               12299
                                1901                2886
                                602                  182
                                1702                   1
                                1703                   1

In [21]:
def get_metrics(df=df_events, eventId=None, subEventId=None, tags=[]):
    '''
    Takes a Wyscout Event ID and/or Sub Event ID, plus a list of Wyscout event tags
    Returns a dataframe of events where all tags in the list were found
    '''

    df_events_meeting_criteria = filter_by_event_or_sub_event(eventId=eventId, subEventId=subEventId)

    for tag in tags:
        tag_name = wyscout_tags_dict[tag]
        df_events_meeting_criteria[tag_name] = df_events_meeting_criteria['tags'].apply(lambda x: {'id':tag} in x)
        df_events_meeting_criteria = df_events_meeting_criteria[df_events_meeting_criteria[tag_name]==True]

    return df_events_meeting_criteria

In [22]:
get_metrics(tags=[1401]).groupby(['subEventName']).count()

,level_0,level_1,eventId,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,interception
subEventName,,,,,,,,,,,,,,
Acceleration,1161,1161,1161,1161,1161,1161,1161,1161,1161,1161,1161,1161,1161,1161
Clearance,30200,30200,30200,30200,30200,30200,30200,30200,30200,30200,30200,30200,30200,30200
Cross,158,158,158,158,158,158,158,158,158,158,158,158,158,158
Ground attacking duel,6519,6519,6519,6519,6519,6519,6519,6519,6519,6519,6519,6519,6519,6519
Ground loose ball duel,426,426,426,426,426,426,426,426,426,426,426,426,426,426
Hand pass,1838,1838,1838,1838,1838,1838,1838,1838,1838,1838,1838,1838,1838,1838
Head pass,23513,23513,23513,23513,23513,23513,23513,23513,23513,23513,23513,23513,23513,23513
High pass,2674,2674,2674,2674,2674,2674,2674,2674,2674,2674,2674,2674,2674,2674
Launch,3129,3129,3129,3129,3129,3129,3129,3129,3129,3129,3129,3129,3129,3129


In [23]:
def summarise_events_by_player(df_of_specific_events, plain_english_label=False):
    
    df_grouped = df_of_specific_events[['playerId', 'id']].groupby('playerId').count()
    df_grouped.columns = [plain_english_label]

    return df_grouped

In [24]:
@dataclass
class PlayerMetric:
    '''Class for storing a complete definition of a player metric.'''
    metric_name: str
    eventId: int
    subEventId: int
    tags: list
    fill_na_zero: bool


In [25]:
metrics_definitions = [ 
    PlayerMetric('ground_duels_won_by_anticipation', 1, 12, [601, 1801], True), 
    PlayerMetric('interceptions', None, None, [1401], True)
]

df_player_aggregates = pd.DataFrame()

for definition in metrics_definitions:
    df_each_event = get_metrics(df=df_events, eventId=definition.eventId, subEventId=definition.subEventId, tags=definition.tags)
    df_player_aggregates = pd.concat([df_player_aggregates, summarise_events_by_player(df_each_event, definition.metric_name)], axis=1)

In [26]:
df_player_aggregates

,ground_duels_won_by_anticipation,interceptions
playerId,,
0,1106.0,240.0
36,1.0,54.0
38,2.0,21.0
45,1.0,17.0
48,33.0,211.0
...,...,...
520617,NaN,7.0
521939,NaN,1.0
523089,NaN,10.0


In [27]:
df_all_players = df_players[['wyId', 'shortName', 'firstName', 'lastName', 'currentTeamId', 'role_name']]
df_all_players

,wyId,shortName,firstName,lastName,currentTeamId,role_name
0,32777,H. Tekin,Harun,Tekin,4502,Goalkeeper
1,393228,M. Sarr,Malang,Sarr,3775,Defender
2,393230,O. Mandanda,Over,Mandanda,3772,Goalkeeper
3,32793,A. N'Diaye,Alfred John Momar,N'Diaye,683,Midfielder
4,393247,I. Konat\u00e9,Ibrahima,Konat\u00e9,2975,Defender
...,...,...,...,...,...,...
3598,120839,A. Ma\u00e2loul,Ali,Ma\u00e2loul,16041,Defender
3599,114736,C. C\u00e1ceda,Carlos Alberto,C\u00e1ceda Oyaguez,15591,Goalkeeper
3600,114908,M. Araujo,Miguel Gianpierre,Araujo Blanco,12072,Defender
3601,285583,A. Tagnaouti,Ahmed Reda,Tagnaouti,16183,Goalkeeper


In [28]:
df_all_players_with_minutes = df_all_players.merge(df_minutes_season, how='left', left_on='wyId', right_on='playerId')
df_all_players_with_minutes['minutesPlayed'] = df_all_players_with_minutes['minutesPlayed'].fillna(0)
df_all_players_with_minutes['90s'] = df_all_players_with_minutes['minutesPlayed'] / 90
df_all_players_with_minutes = df_all_players_with_minutes[df_all_players_with_minutes['90s']>=MINIMUM_90s]
df_all_players_with_minutes.sort_values('minutesPlayed', ascending=False)

,wyId,shortName,firstName,lastName,currentTeamId,role_name,minutesPlayed,90s
2915,8726,A. Begovi\u0107,Asmir,Begovi\u0107,1659,Goalkeeper,3612.0,40.133333
1540,20635,F. Acerbi,Francesco,Acerbi,3315,Defender,3591.0,39.900000
743,8653,H. Maguire,Harry,Maguire,1631,Defender,3586.0,39.844444
1794,350999,G. Donnarumma,Gianluigi,Donnarumma,3157,Goalkeeper,3583.0,39.811111
801,10131,J. Pickford,Jordan,Pickford,1623,Goalkeeper,3575.0,39.722222
...,...,...,...,...,...,...,...,...
90,99633,I. Lewczuk,Igor,Lewczuk,3772,Defender,457.0,5.077778
413,364870,Adri\u00e1n Di\u00e9guez,Adri\u00e1n,Di\u00e9guez Grande,696,Defender,452.0,5.022222
562,366542,J. Reine-Adelaide,Jeff,Reine-Adelaide,3799,Midfielder,451.0,5.011111
2697,658,S. Armenteros,Kristiano Samuel,Armenteros Nunez Mendoza Jansson,8113,Forward,451.0,5.011111


In [44]:
df_all_players_with_minutes_and_metrics = df_all_players_with_minutes.merge(df_player_aggregates, how='left', left_on='wyId', right_index=True)

for definition in metrics_definitions:
    if definition.fill_na_zero:
        df_all_players_with_minutes_and_metrics[definition.metric_name] = df_all_players_with_minutes_and_metrics[definition.metric_name].fillna(0)

df_all_players_with_minutes_and_metrics

,wyId,shortName,firstName,lastName,currentTeamId,role_name,minutesPlayed,90s,ground_duels_won_by_anticipation,interceptions
1,393228,M. Sarr,Malang,Sarr,3775,Defender,1379.0,15.322222,6.0,71.0
4,393247,I. Konat\u00e9,Ibrahima,Konat\u00e9,2975,Defender,1267.0,14.077778,9.0,72.0
6,36,T. Alderweireld,Toby,Alderweireld,1624,Defender,1238.0,13.755556,1.0,54.0
7,48,J. Vertonghen,Jan,Vertonghen,1624,Defender,3343.0,37.144444,33.0,211.0
8,229427,A. Djiku,Alexander,Djiku,3783,Defender,2613.0,29.033333,34.0,170.0
...,...,...,...,...,...,...,...,...,...,...
3402,292654,M. Diakhaby,Mouctar,Diakhaby,674,Defender,765.0,8.500000,5.0,55.0
3408,325781,J. Lea Siliki,James Edward,Lea Siliki,3774,Midfielder,1820.0,20.222222,4.0,53.0
3409,326946,R. Coly,Racine,Coly,3775,Defender,639.0,7.100000,3.0,42.0
3423,393214,V. Rosier,Valentin,Rosier,3804,Defender,2881.0,32.011111,11.0,193.0


In [45]:
for definition in metrics_definitions:
    column_name_per_90 = definition.metric_name + '_p90'
    df_all_players_with_minutes_and_metrics[column_name_per_90] = df_all_players_with_minutes_and_metrics[definition.metric_name] / df_all_players_with_minutes_and_metrics['90s']

In [46]:
df_all_players_with_minutes_and_metrics

,wyId,shortName,firstName,lastName,currentTeamId,role_name,minutesPlayed,90s,ground_duels_won_by_anticipation,interceptions,ground_duels_won_by_anticipation_p90,interceptions_p90
1,393228,M. Sarr,Malang,Sarr,3775,Defender,1379.0,15.322222,6.0,71.0,0.391588,4.633793
4,393247,I. Konat\u00e9,Ibrahima,Konat\u00e9,2975,Defender,1267.0,14.077778,9.0,72.0,0.639305,5.114444
6,36,T. Alderweireld,Toby,Alderweireld,1624,Defender,1238.0,13.755556,1.0,54.0,0.072698,3.925687
7,48,J. Vertonghen,Jan,Vertonghen,1624,Defender,3343.0,37.144444,33.0,211.0,0.888424,5.680526
8,229427,A. Djiku,Alexander,Djiku,3783,Defender,2613.0,29.033333,34.0,170.0,1.171068,5.855339
...,...,...,...,...,...,...,...,...,...,...,...,...
3402,292654,M. Diakhaby,Mouctar,Diakhaby,674,Defender,765.0,8.500000,5.0,55.0,0.588235,6.470588
3408,325781,J. Lea Siliki,James Edward,Lea Siliki,3774,Midfielder,1820.0,20.222222,4.0,53.0,0.197802,2.620879
3409,326946,R. Coly,Racine,Coly,3775,Defender,639.0,7.100000,3.0,42.0,0.422535,5.915493
3423,393214,V. Rosier,Valentin,Rosier,3804,Defender,2881.0,32.011111,11.0,193.0,0.343631,6.029157


In [47]:
# https://en.wikipedia.org/wiki/Weighted_product_model

# How to pick best across multiple criteria?

In [48]:
# swap below to single quotes
# "get" in the lambda

In [49]:
# calculate possession per player

def build_posession_dataframe():
    possession_dict = {}
    #for every row in the dataframe
    for i, row in tqdm(df_minutes_per_game.iterrows()):
        #take player id, team id and match id, minute in and minute out
        player_id, team_id, match_id = row['playerId'], row['teamId'], row['matchId']
        #create a key in dictionary if player encounterd first time
        if not str(player_id) in possession_dict.keys():
            possession_dict[str(player_id)] = {'team_passes': 0, 'all_passes' : 0}
        seconds_in = row['player_in_min'] * 60
        seconds_out = row['player_out_min'] * 60

        #get the dataframe of events from the game
        match_df = df_events.loc[df_events['matchId'] == match_id].copy()
        #add to 2H the highest value of 1H

        match_df.loc[match_df['matchPeriod'] == '2H', 'eventSec'] = match_df.loc[match_df['matchPeriod'] == '2H', 'eventSec'] + match_df.loc[match_df['matchPeriod'] == '1H']['eventSec'].iloc[-1]
        #take all events from this game and this period
        player_in_match_df = match_df.loc[match_df['eventSec'] > seconds_in].loc[match_df['eventSec'] <= seconds_out]
        #take all passes and won duels as described
        all_passes = player_in_match_df.loc[player_in_match_df['eventName'].isin(['Pass', 'Duel'])]
        #adjusting for no passes in this period (Tuanzebe)
        if len(all_passes) > 0:
            #removing lost air duels
            no_contact = all_passes.loc[all_passes['subEventName'].isin(['Air duel', 'Ground defending duel','Ground loose ball duel'])].loc[all_passes.apply(lambda x:{'id':701} in x.tags, axis = 1)]
            all_passes = all_passes.drop(no_contact.index)
        #take team passes
        team_passes = all_passes.loc[all_passes['teamId'] == team_id]
        #append it {player id: {team passes: sum, all passes : sum}}
        possession_dict[str(player_id)]['team_passes'] += len(team_passes)
        possession_dict[str(player_id)]['all_passes'] += len(all_passes)

    #calculate possession for each player
    percentage_dict = {key: value['team_passes']/value['all_passes'] if value['all_passes'] > 0 else 0 for key, value in possession_dict.items()}
    #create a dataframe
    percentage_df = pd.DataFrame(percentage_dict.items(), columns = ['playerId', 'possession'])
    percentage_df['playerId'] = percentage_df['playerId'].astype(int)

    percentage_df.to_csv('percentage_possession.csv')

    return percentage_df

In [50]:
# retrieve possession per player if already exists on disk, else build it
try:
    df_possession_per_player = pd.read_csv('percentage_possession.csv', index_col=0)
except FileNotFoundError:
    df_possession_per_player = build_posession_dataframe()

In [51]:
df_possession_per_player

,playerId,possession
0,9206,0.446999
1,9127,0.439018
2,93,0.438288
3,9179,0.471322
4,10108,0.433335
...,...,...
2565,421809,0.608048
2566,37734,0.597884
2567,25519,0.409378
2568,294697,0.648485


In [52]:
df_all_players_with_minutes_and_metrics_and_possession = df_all_players_with_minutes_and_metrics.merge(df_possession_per_player, how='left', left_on='wyId', right_on='playerId')

In [87]:
for definition in metrics_definitions:
    column_name_possession_adjusted_p90 = definition.metric_name + '_p90_padj'
    df_all_players_with_minutes_and_metrics_and_possession[column_name_possession_adjusted_p90] = df_all_players_with_minutes_and_metrics_and_possession[definition.metric_name + '_p90'] / df_all_players_with_minutes_and_metrics_and_possession['possession']


In [88]:
df_all_players_with_minutes_and_metrics_and_possession[df_all_players_with_minutes_and_metrics_and_possession['role_name']==TARGET_ROLE].sort_values('ground_duels_won_by_anticipation_p90_padj', ascending=False).head(30)

,wyId,shortName,firstName,lastName,currentTeamId,role_name,minutesPlayed,90s,ground_duels_won_by_anticipation,interceptions,ground_duels_won_by_anticipation_p90,interceptions_p90,playerId,possession,ground_duels_won_by_anticipation_p90_padj,interceptions_p90_padj,ground_duels_won_by_anticipation_p90_padj_rank,interceptions_p90_padj_rank
1234,3797,V\u00edctor S\u00e1nchez,V\u00edctor,S\u00e1nchez Mata,691,Midfielder,2535.0,28.166667,32.0,176.0,1.136095,6.248521,3797,0.451629,2.515547,13.835506,0.986287,0.911630
195,3566,Camacho,Ignacio,Camacho Barnola,2451,Midfielder,991.0,11.011111,12.0,94.0,1.089808,8.536831,3566,0.485355,2.245386,17.588855,0.978162,0.993906
1422,25995,B. Stambouli,Benjamin,Stambouli,2449,Midfielder,2283.0,25.366667,27.0,155.0,1.064389,6.110381,25995,0.483211,2.202740,12.645357,0.976130,0.859319
1906,339789,Mikel Merino,Mikel,Merino Zaz\u00f3n,687,Midfielder,1419.0,15.766667,14.0,105.0,0.887949,6.659619,339789,0.437200,2.030989,15.232417,0.969528,0.963433
1206,22158,I. Cofie,Isaac,Cofie,3193,Midfielder,622.0,6.911111,6.0,36.0,0.868167,5.209003,22158,0.454720,1.909233,11.455399,0.962417,0.776028
1746,8143,M. Schneiderlin,Morgan,Schneiderlin,1623,Midfielder,2224.0,24.711111,21.0,152.0,0.849820,6.151079,8143,0.454516,1.869726,13.533258,0.960386,0.899949
672,407699,D. Avounou,Bel Durel,Avounou,3783,Midfielder,665.0,7.388889,6.0,46.0,0.812030,6.225564,407699,0.441731,1.838289,14.093550,0.957847,0.921788
1286,221376,M. Steinmann,Ville Matti,Steinmann,2450,Midfielder,610.0,6.777778,6.0,47.0,0.885246,6.934426,221376,0.551656,1.604706,12.570197,0.934992,0.855764
709,14781,L. Bender,Lars,Bender,2446,Midfielder,1638.0,18.200000,15.0,96.0,0.824176,5.274725,14781,0.523733,1.573657,10.071408,0.930929,0.669883
649,45579,Everton Luiz,Everton Luiz,Guimaraes Bilher,3204,Midfielder,679.0,7.544444,5.0,26.0,0.662739,3.446244,45579,0.423531,1.564795,8.136936,0.928898,0.522092


In [89]:
def calculate_percentiles(position=None, df=df_all_players_with_minutes_and_metrics_and_possession):    
    if position:
        df = df[df['role_name']==position].copy(deep=True)

    for definition in metrics_definitions:
        column_name_p90_padj_rank = definition.metric_name + '_p90_padj_rank'
        df[column_name_possession_adjusted_p90] = df[definition.metric_name + '_p90_padj'].rank(pct=True)

    return df

In [94]:
calculate_percentiles(position='Midfielder').sort_values('ground_duels_won_by_anticipation_p90_padj_rank', ascending=False)

,wyId,shortName,firstName,lastName,currentTeamId,role_name,minutesPlayed,90s,ground_duels_won_by_anticipation,interceptions,ground_duels_won_by_anticipation_p90,interceptions_p90,playerId,possession,ground_duels_won_by_anticipation_p90_padj,interceptions_p90_padj,ground_duels_won_by_anticipation_p90_padj_rank,interceptions_p90_padj_rank
1234,3797,V\u00edctor S\u00e1nchez,V\u00edctor,S\u00e1nchez Mata,691,Midfielder,2535.0,28.166667,32.0,176.0,1.136095,6.248521,3797,0.451629,2.515547,1.000000,0.986287,0.911630
195,3566,Camacho,Ignacio,Camacho Barnola,2451,Midfielder,991.0,11.011111,12.0,94.0,1.089808,8.536831,3566,0.485355,2.245386,0.998590,0.978162,0.993906
1422,25995,B. Stambouli,Benjamin,Stambouli,2449,Midfielder,2283.0,25.366667,27.0,155.0,1.064389,6.110381,25995,0.483211,2.202740,0.997179,0.976130,0.859319
1906,339789,Mikel Merino,Mikel,Merino Zaz\u00f3n,687,Midfielder,1419.0,15.766667,14.0,105.0,0.887949,6.659619,339789,0.437200,2.030989,0.995769,0.969528,0.963433
1206,22158,I. Cofie,Isaac,Cofie,3193,Midfielder,622.0,6.911111,6.0,36.0,0.868167,5.209003,22158,0.454720,1.909233,0.994358,0.962417,0.776028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,135913,Rony Lopes,Marcos Paulo,Mesquita Lopes,19830,Midfielder,2595.0,28.833333,0.0,57.0,0.000000,1.976879,135913,0.500335,0.000000,0.068406,0.099035,0.203657
718,14804,J. B\u0142aszczykowski,Jakub,B\u0142aszczykowski,2451,Midfielder,534.0,5.933333,0.0,20.0,0.000000,3.370787,14804,0.482586,0.000000,0.068406,0.099035,0.436770
348,70403,Adrien Silva,Adrien Sebastian,Perruchet Silva,1631,Midfielder,797.0,8.855556,0.0,32.0,0.000000,3.613551,70403,0.515497,0.000000,0.068406,0.099035,0.439817
349,4879,Carles Gil,Carles,Gil de Pareja Vicent,677,Midfielder,490.0,5.444444,0.0,18.0,0.000000,3.306122,4879,0.479669,0.000000,0.068406,0.099035,0.429152
